<p align="center">
<img src="img/lasair.png" alt="drawing" width="50"/>
</p>
<h1 align="center">  Lasair API Recipes </h1>


---

## What you'll learn

[TBD]

## Pre-requesites

### **Have you got a Lasair Token?**

**_Why do I need a token to stream public alerts?_** Because without authentication systems, bad actors can mess with our servers. But creating a Lasair profile is **Free and Public** for everyone. You can do this on the [Lasair Platform](http://lasair.lsst.ac.uk/) (go to Sing-In page).

Then you will be able to **see and copy your token** from your [profile](http://lasair.lsst.ac.uk/profile) page. 

**You should not share your token or save it in public repositories e.g. GitHub**.  One of the easies things to do so you don't have to keep pasting it, is to save it on your local machine in your shell environment. For example add this to your `bashrc` or `bash_profile`

```bash
export LASAIR_LSST_TOKEN=blablablablablYOUROTKENHEREblablabla
```

### **Have you installed the `lasair` client ?**

You can do this through pip!

```bash
pip install lasair
```

---


<h2 align="center">  1. Cone Search </h1>




In [2]:
from lasair import lasair_client as lapi
import os

If you have set your environment variable you can get it into python very easily!   

In [ ]:
API_TOKEN = os.getenv('LASAIR_LSST_TOKEN')

if API_TOKEN is None:
    print("No Token found. Check Spelling. Note that if you have just added your token to your environment variables, you may need to restart your terminal so your shell settings are reloaded. ")

Now we create our lasair API client instance, it takes in our token and the location of the Lasair API on the internet (endpoint).

In [ ]:
L = lapi(API_TOKEN, 
         endpoint="https://lasair-lsst-dev.lsst.ac.uk/api/",
         #PROD endpoint='https://api.lasair.lsst.ac.uk/api'
         )

Let's pick a random sky location, and set our search radius to 15 arcseconds

In [82]:
ra = 149.164175
dec= 2.649324
radius = 15

There are **3 types of Cone Searches**:
- Count: Number of alerts found within the radius
- Nearest: Which object is nearest (but still within the radius! If radius is too small this can be 0)
- All: Every alert that fulfill the criteria


### Count


In [83]:
result = L.cone(ra, 
                dec, 
                radius, 
                requestType='count'
                )

print('Found %d at radius %f' % (result['count'], radius))

Found 4 at radius 15.000000


### Nearest

In [84]:
result = L.cone(ra, dec, radius, requestType='nearest')
if 'nearest' in result:
    obj = result['nearest']
    print('Found %s at separation %.2f with radius %f' % (obj['object'], obj['separation'], radius))
else:
    print('No object found at radius %f' % radius)

Found 313998569323888791 at separation 0.00 with radius 15.000000


### ALL

In [87]:
print('At radius %f' % radius)
result = L.cone(ra, dec, radius, requestType='all')
if 'object' in result['nearest']:
    print('nearest ', result['nearest']['object'], \
        'at separation %.2f' % result['nearest']['separation'])
    objects = result['objects']
    for obj in objects:
        print(obj['object'], "%.2f" % obj['separation'])
else:
    print('no object found')

At radius 15.000000
nearest  313998569323888791 at separation 0.00
313998569323888791 0.00
169883381784903846 0.29
314011776099811451 1.63
314003015436075376 3.45



<h2 align="center">  2. Query by diaObjectId </h1>

In [88]:
result = L.object(313963406691402058, lasair_added=True)

<h2 align="center">  3. Advanced Queries </h1>

You can build advanced SQL-like queries for more refinded searches. In this example we will look at finding objects based on their [Sherlock](https://lasair.readthedocs.io/en/main/core_functions/sherlock.html#sherlock--sky-context-) Classification, which is based on catalogue cross-matching. 


### Select your data
First let's define the **columns we want to get out of our query**: here it will be the `diaObjectId`, Ra and Dec.

Since we're doing an SQL query, we need to know the correct name for the table and columns, which you can find on the [Lasair Website Schema Browser](https://lasair-lsst-dev.lsst.ac.uk/schema/) 


![schema_browser](img/schema_browser.png)

In [89]:
selected = 'objects.diaObjectId, objects.ra, objects.decl'

### Select your conditions

The next thing we want to do is format our conditions, where we want the data **only if Sherlock said it could be a SN** (i.e. near a galaxy but not at the nucleus).

Scrollking down on our Schema Broswer we can see there is a `sherlock_classification` with a `classification` column. We'll need that == `SN` 

![schema_browser](img/schema_browser_sherlock.png)


In [90]:
conditions = 'classification="SN"'

*[FAQ]: Why is it not `sherlock_classification.classification="SN"`?*

### List the tables needed for your query

In [91]:
tables = 'objects,sherlock_classifications'

### Call your query!

You'll notice here we also set a limit of 8, because a lot of objects actually match this condition.


In [92]:
results = L.query(selected, tables, conditions, limit = 8)

In [93]:
results

[{'diaObjectId': 169298433161560316,
  'ra': 10.145744291286427,
  'decl': -44.754963238154986},
 {'diaObjectId': 169298433201930256,
  'ra': 10.250262019119214,
  'decl': -43.299100965937846},
 {'diaObjectId': 169298436674813978,
  'ra': 8.796281610710041,
  'decl': -43.98532171023203},
 {'diaObjectId': 169298436985192488,
  'ra': 8.313652586179952,
  'decl': -43.036609073180145},
 {'diaObjectId': 169298437172363418,
  'ra': 9.666946696565965,
  'decl': -45.64004534870912},
 {'diaObjectId': 169298437307105370,
  'ra': 10.041701764645088,
  'decl': -45.53332938361457},
 {'diaObjectId': 169298437505810433,
  'ra': 7.8915515318678215,
  'decl': -44.500186689763105},
 {'diaObjectId': 169298437514723368,
  'ra': 8.631704335763443,
  'decl': -42.617216230724495}]

<h2 align="center">  4. Querying an Annotator </h1>

[TBD]